In [86]:
import plotly
import matplotlib.pyplot as plt
import dill
import sympy
from tqdm import tqdm
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
import gmsh
import numpy as np

### Load precomputed integrals

In [87]:
weak_form_functional_lambdified_quadrature = dill.load(
    open(
        "../calculations/bilinear_lagrange_weak_form_functional", "rb"
    )
)

weak_form_right_part_lambdified_quadrature = dill.load(    
    open("../calculations/bilinear_lagrange_weak_form_right_part", "rb")
)

interpolation_function_lambdified = dill.load(
    open("../calculations/bilinear_lagrange_basis", "rb")
)

mapping_function_lambdified = dill.load(
    open("../calculations/bilinear_lagrange_mapping_function", "rb")
)

### Initialize the domain

In [122]:
gmsh.initialize()
gmsh.model.add("quad_mesh")

# Define points
point_mesh_size = 0.03
point_1 = gmsh.model.geo.addPoint(0, 0, 0, point_mesh_size)
point_2 = gmsh.model.geo.addPoint(1, 1, 0, point_mesh_size)
point_3 = gmsh.model.geo.addPoint(-1, 1, 0, point_mesh_size)
point_4 = gmsh.model.geo.addPoint(-1, -1, 0, point_mesh_size)
point_5 = gmsh.model.geo.addPoint(1, -1, 0, point_mesh_size)

# Define circles
arc_1 = gmsh.model.geo.addLine(point_2, point_3)
arc_2 = gmsh.model.geo.addLine(point_3, point_4)
arc_3 = gmsh.model.geo.addLine(point_4, point_5)
arc_4 = gmsh.model.geo.addLine(point_5, point_2)


# arc_1 = gmsh.model.geo.addCircleArc(point_2, point_1, point_3)
# arc_2 = gmsh.model.geo.addCircleArc(point_3, point_1, point_4)
# arc_3 = gmsh.model.geo.addCircleArc(point_4, point_1, point_5)
# arc_4 = gmsh.model.geo.addCircleArc(point_5, point_1, point_2)

# Define line loop
circle = gmsh.model.geo.addCurveLoop([arc_1, arc_2, arc_3, arc_4])

# Define plane surface
plane = gmsh.model.geo.addPlaneSurface([circle])

gmsh.model.geo.synchronize()
# gmsh.option.setNumber(name="Mesh.Smoothing", value=30)

gmsh.model.mesh.setTransfiniteCurve(arc_1, 15)
gmsh.model.mesh.setTransfiniteCurve(arc_2, 15)
gmsh.model.mesh.setTransfiniteCurve(arc_3, 15)
gmsh.model.mesh.setTransfiniteCurve(arc_4, 15)
gmsh.model.mesh.setTransfiniteSurface(plane)
gmsh.model.mesh.setRecombine(2, plane)

physical_group_curves_tag = gmsh.model.addPhysicalGroup(1, [arc_1, arc_2, arc_3, arc_4], name="Boundary curves")
physical_group_surface_tag = gmsh.model.addPhysicalGroup(2, [plane], name="Surface")


# point_tag = point_1  # replace with your point's tag
# attractor_field = gmsh.model.mesh.field.add("Attractor")
# gmsh.model.mesh.field.setNumbers(attractor_field, "NodesList", [point_tag])
# threshold_field = gmsh.model.mesh.field.add("Threshold")
# gmsh.model.mesh.field.setNumber(threshold_field, "IField", attractor_field)
# gmsh.model.mesh.field.setNumber(threshold_field, "LcMin", 0.0001)
# gmsh.model.mesh.field.setNumber(threshold_field, "LcMax", 0.1)
# gmsh.model.mesh.field.setNumber(threshold_field, "DistMin", 0.0001)
# gmsh.model.mesh.field.setNumber(threshold_field, "DistMax", 0.4)
# gmsh.model.mesh.field.setAsBackgroundMesh(threshold_field)



gmsh.model.mesh.setRecombine(2, plane)
gmsh.model.mesh.generate(2)
gmsh.fltk.run()

# Quads

In [123]:
boundary_node_tags, boundary_node_coords = gmsh.model.mesh.getNodesForPhysicalGroup(1, physical_group_curves_tag)
surface_node_tags, points = gmsh.model.mesh.getNodesForPhysicalGroup(2, physical_group_surface_tag)
points = points.reshape(-1, 3)[:, :-1]
boundary_node_tags = boundary_node_tags - 1

In [124]:
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()
quads = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 3][0]
quads = quads.reshape(-1, 4)
quads = quads - 1

In [125]:
vertex_marker_is_boundary = np.zeros(surface_node_tags.shape[0])
vertex_marker_is_boundary[boundary_node_tags] = 1

In [126]:
x, y = sympy.symbols("x y")

In [127]:
F = sympy.Float(1)
right_vals = np.zeros((points.shape[0]))
right_vals[:] = sympy.lambdify((x, y), F)(*points.T)

G = sympy.Piecewise((1, sympy.Eq(x, 1)), (0, True))*0
right_vals_G = np.zeros((points.shape[0]))
right_vals_G[:] = sympy.lambdify((x, y), G)(*points.T)

In [128]:
matrix = lil_matrix((points.shape[0], points.shape[0]))

b = np.zeros(points.shape[0])

for idx, element in enumerate(tqdm(quads)):
    x1, x2, x3, x4 = points[element[0], 0], points[element[1], 0], points[element[2], 0], points[element[3], 0]
    y1, y2, y3, y4 = points[element[0], 1], points[element[1], 1], points[element[2], 1], points[element[3], 1]


    FF = weak_form_functional_lambdified_quadrature(x1, y1, x2, y2, x3, y3, x4, y4)
    ff = weak_form_right_part_lambdified_quadrature(x1, y1, x2, y2, x3, y3, x4, y4,
                                                    right_vals[element[0]],
                                                    right_vals[element[1]],
                                                    right_vals[element[2]],
                                                    right_vals[element[3]],
                                                    )

    for trial_dof_idx in range(4):
        
        if vertex_marker_is_boundary[element[trial_dof_idx]] == 1:
            
            for test_dof_idx in range(4):
                b[element[test_dof_idx]] -= right_vals_G[element[trial_dof_idx]]*FF[trial_dof_idx, test_dof_idx]
                
            b[element[trial_dof_idx]] = right_vals_G[element[trial_dof_idx]]
                        
        else:
            for test_dof_idx in range(4):
                matrix[element[trial_dof_idx], element[test_dof_idx]] += FF[trial_dof_idx, test_dof_idx]

            b[element[trial_dof_idx]] -= ff[trial_dof_idx, 0]
            
            
bv = np.where(vertex_marker_is_boundary)[0]

for b_ in tqdm(bv):    
    matrix[b_, :] = 0
    matrix[:, b_] = 0
    matrix[b_, b_] = 1
    b[b_] = right_vals_G[b_]

  0%|          | 0/196 [00:00<?, ?it/s]

100%|██████████| 56/56 [00:00<00:00, 4686.65it/s]


In [129]:
# plt.imshow(matrix.toarray() != 0, cmap="gray")

In [130]:
F = spsolve(matrix.tocsc(), b)

In [131]:
param = np.linspace(0, 1, 10)
gridx, gridy = np.meshgrid(param, param, indexing="ij")

gridx = gridx.flatten()
gridy = gridy.flatten()

In [132]:
global_x, global_y, global_z = [], [], []

for element in quads:
    
    x1, x2, x3, x4 = points[element[0], 0], points[element[1], 0], points[element[2], 0], points[element[3], 0]
    y1, y2, y3, y4 = points[element[0], 1], points[element[1], 1], points[element[2], 1], points[element[3], 1]

    physical_x, physical_y = mapping_function_lambdified(gridx, gridy, x1, y1, x2, y2, x3, y3, x4, y4)[:, 0]
    
    physical_z = interpolation_function_lambdified(
                                        gridx, gridy,
                                        x1, y1, 
                                        x2, y2, 
                                        x3, y3,
                                        x4, y4,
                                        F[element[0]], 
                                        F[element[1]], 
                                        F[element[2]],
                                        F[element[3]],                                        
                                        )
    
    global_x.append(physical_x)
    global_y.append(physical_y)
    global_z.append(physical_z)

global_x = np.concatenate(global_x)
global_y = np.concatenate(global_y)
global_z = np.concatenate(global_z)

In [133]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=gx, 
                                y=gy,
                                z=gz, 
                                )
        for gx, gy, gz in zip(global_x.reshape(-1, gridx.shape[0]), 
                              global_y.reshape(-1, gridx.shape[0]), 
                              global_z.reshape(-1, gridx.shape[0]))
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()

In [135]:
# plot_data = [
#     plotly.graph_objects.Mesh3d(
#                                 x=global_x, 
#                                 y=global_y,
#                                 z=global_z,
#                                 intensity=global_z
#                                 )        
#     ]
# fig = plotly.graph_objects.Figure(data=plot_data)
# fig.update_layout()